<a href="https://colab.research.google.com/github/Pelmenoff/data_science/blob/main/hw9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras-tuner --upgrade

In [2]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
# Завантаження даних
(x, y), _ = fashion_mnist.load_data()

# Спочатку розділяємо дані на 70% тренувальні і 30% для тестування та валідації
x_train, x_test_raw, y_train, y_test_raw = train_test_split(x, y, train_size=0.7, random_state=555)

# Тепер розділимо ці 30% на 20% для тесту і 15% для валідації
# 20% від загального числа = 20/30 частини від x_test_raw та y_test_raw
test_size = 20 / 30
x_val, x_test, y_val, y_test = train_test_split(x_test_raw, y_test_raw, test_size=test_size, random_state=555)

print("Train data size", x_train.shape)
print("Test data size", x_test.shape)
print("Validation data size", x_val.shape)

# Масштабування зображень до діапазону [0, 1]
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Розширення розмірності (додавання каналу)
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot кодування міток
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

Train data size (42000, 28, 28)
Test data size (12000, 28, 28)
Validation data size (6000, 28, 28)


In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))

    for i in range(hp.Int('num_layers', 2, 6)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=64, max_value=1024, step=64), activation='relu'))
        model.add(BatchNormalization())

    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer=Nadam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    hyperband_iterations=2
)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val))

# Получение лучших гиперпараметров и модели
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val))

Trial 73 Complete [00h 33m 18s]
val_accuracy: 0.8981666564941406

Best val_accuracy So Far: 0.9010000228881836
Total elapsed time: 04h 16m 23s

Search: Running Trial #74

Value             |Best Value So Far |Hyperparameter
2                 |6                 |num_layers
704               |960               |units_0
256               |576               |units_1
0.00027728        |0.00019832        |learning_rate
64                |64                |units_2
768               |128               |units_3
960               |704               |units_4
256               |640               |units_5
50                |50                |tuner/epochs
17                |17                |tuner/initial_epoch
2                 |3                 |tuner/bracket
2                 |3                 |tuner/round
0070              |0048              |tuner/trial_id

Epoch 18/50
1313/1313 [==============================] - 22s 15ms/step - loss: 0.2551 - accuracy: 0.9047 - val_loss: 0.3195 - val_accu

In [4]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28, 1)))

model.add(Dense(704))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(768))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(960))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(10, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 704)               552640    
                                                                 
 batch_normalization (Batch  (None, 704)               2816      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 256)               180480    
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 64)                1

In [5]:
# Компільовання моделі
model.compile(optimizer=Nadam(learning_rate=0.0002),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [6]:
# Тренування моделі
history = model.fit(x_train, y_train, batch_size=32,
                    epochs=50, validation_data=(x_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/50
1313/1313 [==============================] - 66s 40ms/step - loss: 0.5153 - accuracy: 0.8181 - val_loss: 0.4065 - val_accuracy: 0.8502 - lr: 2.0000e-04
Epoch 2/50
1313/1313 [==============================] - 52s 40ms/step - loss: 0.3804 - accuracy: 0.8604 - val_loss: 0.3380 - val_accuracy: 0.8730 - lr: 2.0000e-04
Epoch 3/50
1313/1313 [==============================] - 53s 40ms/step - loss: 0.3393 - accuracy: 0.8751 - val_loss: 0.3431 - val_accuracy: 0.8775 - lr: 2.0000e-04
Epoch 4/50
1313/1313 [==============================] - 51s 39ms/step - loss: 0.3104 - accuracy: 0.8850 - val_loss: 0.3226 - val_accuracy: 0.8835 - lr: 2.0000e-04
Epoch 5/50
1313/1313 [==============================] - 51s 39ms/step - loss: 0.2854 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.8768 - lr: 2.0000e-04
Epoch 6/50
1313/1313 [==============================] - 51s 39ms/step - loss: 0.2671 - accuracy: 0.8989 - val_loss: 0.3346 - val_accuracy: 0.8800 - lr: 2.0000e-04
Epoch 7/50
1313/1313 [

In [7]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}, accuracy: {test_acc}')

375/375 [==============================] - 3s 8ms/step - loss: 0.2876 - accuracy: 0.9110
Test loss: 0.2875920534133911, accuracy: 0.9110000133514404
